In [7]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load your saved model
model = load_model("fight_detection_model.h5")

# Parameters
SEGMENT_FRAMES = 10    # number of frames per segment
THRESHOLD = 0.6        # violence probability threshold

def extract_frames_from_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (224,224))
        frame = frame / 255.0
        frames.append(frame)
    cap.release()
    return np.array(frames)

def detect_fights_in_video(video_path):
    frames = extract_frames_from_video(video_path)
    num_frames = len(frames)
    
    # Copy original frames for visualization
    frames_show = (frames * 255).astype(np.uint8).copy()
    
    # Predict in segments
    for start in range(0, num_frames - SEGMENT_FRAMES + 1, SEGMENT_FRAMES):
        segment = frames[start:start + SEGMENT_FRAMES]
        segment_input = np.expand_dims(segment, axis=0)
        pred = model.predict(segment_input)[0]
        violence_prob = pred[0]
        alert = "Fight Detected!" if violence_prob > THRESHOLD else "Safe"
        
        # Overlay alert on segment frames
        for i in range(start, start + SEGMENT_FRAMES):
            frame = frames_show[i]
            color = (0,0,255) if violence_prob > THRESHOLD else (0,255,0)
            cv2.putText(frame, alert, (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
            if violence_prob > THRESHOLD:
                # Draw bounding box around center of frame
                h, w, _ = frame.shape
                cv2.rectangle(frame, (w//4,h//4), (3*w//4,3*h//4), color, 3)
    
    # Play video with overlay
    for frame in frames_show:
        cv2.imshow("Fight Detection", frame)
        if cv2.waitKey(50) & 0xFF == ord('q'):  # 20 FPS approx
            break
    cv2.destroyAllWindows()

# Example usage
video_path = "test4.mp4"  # replace with your video path
detect_fights_in_video(video_path)


1/1 [==============================] - 0s 208ms/step
